# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd 
import sqlite3
from sqlalchemy import create_engine
import joblib
import re
# natural langauge libs
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cnrd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cnrd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cnrd\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cnrd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table("RawData", con=engine, index_col="id")
df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df["message"].values
Y = df.iloc[:,3:].values
categories = df.iloc[:,3:].columns
categories = df.iloc[:,3:].columns

In [4]:
categories

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [15]:
from langdetect import detect
languages = []

for t in X:
    text1 = re.sub(r"[^a-zA-Z]", " ", t.lower())
    try:
        languages.append(detect(text1))
    except:
        pass
languages = set(languages)
print(languages)

#tokenize(X)

{'lt', 'tr', 'it', 'af', 'sk', 'pt', 'nl', 'pl', 'so', 'da', 'no', 'en', 'ro', 'es', 'sq', 'tl', 'fr', 'hr', 'sl', 'sv', 'ca', 'cy', 'id', 'et'}


In [21]:
def tokenize(text):
    # for t in text:
    #     text1 = re.sub(r"[^a-zA-Z0-9]", " ", t.lower())
    #     tokens = word_tokenize(text1)
    #     tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # lemmatizer = WordNetLemmatizer()

    # clean_tokens = []
    # for tok in tokens:
    #     clean_tok = lemmatizer.lemmatize(tok).lower().strip()
    #     clean_tokens.append(clean_tok)

    # Normalize text
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    words = word_tokenize(text)

    # Remove stop words
    words = [w for w in words if w not in stopwords.words(stopwords_langauges)]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for word in words:
        lemmatized_word = lemmatizer.lemmatize(word)
        clean_tokens.append(lemmatized_word)

    return clean_tokens

In [16]:
import pycountry as pc 
languages1 = []
for language in languages:
    language1 = pc.languages.get(alpha_2=language)
    languages1.append(language1.name)
print(languages1)

['Lithuanian', 'Turkish', 'Italian', 'Afrikaans', 'Slovak', 'Portuguese', 'Dutch', 'Polish', 'Somali', 'Danish', 'Norwegian', 'English', 'Romanian', 'Spanish', 'Albanian', 'Tagalog', 'French', 'Croatian', 'Slovenian', 'Swedish', 'Catalan', 'Welsh', 'Indonesian', 'Estonian']


In [17]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [20]:
stopwords_langauges = []
for lan in stopwords.fileids():
    for lan1 in languages1:
        if lan == lan1.lower():
            stopwords_langauges.append(lan)
print(stopwords_langauges)

['danish', 'dutch', 'english', 'french', 'indonesian', 'italian', 'norwegian', 'portuguese', 'romanian', 'spanish', 'swedish', 'turkish']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
# pipeline = Pipeline([
#     ('features', FeatureUnion([
#         ('text_pipeline', Pipeline([
#             ("vect", CountVectorizer(tokenizer=tokenize)),
#             ("tfidf", TfidfTransformer())
#         ])),
#         # ('starting_verb', StartingVerbExtractor())
#     ])),
#     ("clf", MultiOutputClassifier(RandomForestClassifier()))
# ])

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        #('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)),
        ('clf', MultiOutputClassifier(XGBClassifier(early_stopping_rounds=5, eval_metric="mlogloss"), n_jobs=-1))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001D2B650C4C0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=5,
                                                               eval_metric='mlogloss',
                                                               gamma=None,
                                                               gpu_id=None,
                                                               importance_

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
Y_pred = pipeline.predict(X_test)

In [33]:
# for i in range(0, 35):
#     print(f"{i}: {classification_report(Y_test.iloc[:,i], Y_pred[:,i])}")
index = 0
for category in categories:
        print(f"Category: {category} (index = {index})")
        evaluation_report = classification_report(Y_test[:,index], Y_pred[:,index], output_dict=True)
        index += 1
        report = evaluation_report['weighted avg']
        # print(report)
        print(f"Precision: {report['precision']:0.4f}, \tRecall: {report['recall']:0.4f}, \tF1 Score: {report['f1-score']:0.4f}")

Category: related (index = 0)
Precision: 0.8019, 	Recall: 0.8147, 	F1 Score: 0.7933
Category: request (index = 1)
Precision: 0.8880, 	Recall: 0.8952, 	F1 Score: 0.8876
Category: offer (index = 2)
Precision: 0.9893, 	Recall: 0.9945, 	F1 Score: 0.9919
Category: aid_related (index = 3)
Precision: 0.7796, 	Recall: 0.7795, 	F1 Score: 0.7757
Category: medical_help (index = 4)
Precision: 0.9097, 	Recall: 0.9252, 	F1 Score: 0.9107
Category: medical_products (index = 5)
Precision: 0.9542, 	Recall: 0.9607, 	F1 Score: 0.9539
Category: search_and_rescue (index = 6)
Precision: 0.9697, 	Recall: 0.9759, 	F1 Score: 0.9698
Category: security (index = 7)
Precision: 0.9701, 	Recall: 0.9803, 	F1 Score: 0.9724
Category: military (index = 8)
Precision: 0.9631, 	Recall: 0.9691, 	F1 Score: 0.9644
Category: water (index = 9)
Precision: 0.9643, 	Recall: 0.9662, 	F1 Score: 0.9649
Category: food (index = 10)
Precision: 0.9521, 	Recall: 0.9525, 	F1 Score: 0.9523
Category: shelter (index = 11)
Precision: 0.9409, 	R

### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
# parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
#         'clf__estimator__n_estimators': [50, 100, 200],
#         'clf__estimator__min_samples_split': [2, 3, 4],
#     }

parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, Y_train)

KeyboardInterrupt: 

In [24]:
MultiOutputClassifier(estimator=RandomForestClassifier()).get_params().keys()

dict_keys(['estimator__bootstrap', 'estimator__ccp_alpha', 'estimator__class_weight', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__max_samples', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs'])

In [18]:
MultiOutputClassifier(estimator=XGBClassifier()).get_params().keys()

dict_keys(['estimator__objective', 'estimator__use_label_encoder', 'estimator__base_score', 'estimator__booster', 'estimator__colsample_bylevel', 'estimator__colsample_bynode', 'estimator__colsample_bytree', 'estimator__gamma', 'estimator__gpu_id', 'estimator__importance_type', 'estimator__interaction_constraints', 'estimator__learning_rate', 'estimator__max_delta_step', 'estimator__max_depth', 'estimator__min_child_weight', 'estimator__missing', 'estimator__monotone_constraints', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__num_parallel_tree', 'estimator__random_state', 'estimator__reg_alpha', 'estimator__reg_lambda', 'estimator__scale_pos_weight', 'estimator__subsample', 'estimator__tree_method', 'estimator__validate_parameters', 'estimator__verbosity', 'estimator', 'n_jobs'])

In [32]:
parameters = {
        "vect__ngram_range": ((1, 1), (1, 2)),
        "vect__max_df": (0.5, 0.75, 1.0),
        "vect__max_features": (None, 5000, 10000),
        "tfidf__use_idf": (True, False),
        "clf__estimator__min_child_weight": [1, 5, 10],
        "clf__estimator__gamma": [0.5, 1, 1.5, 2, 5],
        "clf__estimator__subsample": [0.6, 0.8, 1.0],
        "clf__estimator__colsample_bytree": [0.6, 0.8, 1.0],
        "clf__estimator__max_depth": [3, 4, 5],
        'clf__estimator__learning_rate': [0.1, 0.01, 0.05],
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, Y_train)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [34]:
joblib.dump(pipeline, "classifier.pkl")

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.